# Saudi Stock Market Analysis
This notebook provides a comprehensive analysis of stock market data for various companies, aiming to uncover patterns, trends, and insights. The analysis is guided by a set of data-driven questions, each designed to provide insights into different aspects of the market, such as price performance, transaction volumes, and market activity.

In [1]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [6]:
file_path = r'C:\Users\kingd\OneDrive\سطح المكتب\Projects\Datathon\advanced rag\\data\Saudi_Stock_11092024.xlsx' 
data = pd.read_excel(file_path)

# Data Preprocessing

In [7]:
print(data.isnull().sum())

Unnamed: 0        0
Stock_Id          0
Company Name      0
52 Weeks Range    0
Par Value         0
Price             0
Change            0
Change (%)        0
Prev. Close       0
Volume            0
Turnover          0
Transactions      0
dtype: int64


In [8]:
print(data.dtypes)

Unnamed: 0          int64
Stock_Id            int64
Company Name       object
52 Weeks Range     object
Par Value          object
Price             float64
Change             object
Change (%)         object
Prev. Close       float64
Volume              int64
Turnover            int64
Transactions        int64
dtype: object


In [9]:
data.describe()

,Unnamed: 0,Stock_Id,Price,Prev. Close,Volume,Turnover,Transactions
count,258.000000,258.000000,258.000000,258.000000,2.580000e+02,2.580000e+02,258.000000
mean,128.500000,4194.794574,52.860426,52.952946,4.803442e+06,2.940089e+07,1872.922481
std,74.622383,2308.426548,89.415242,90.024461,5.598941e+07,9.131151e+07,2763.081675
min,0.000000,1010.000000,0.330000,0.330000,3.450000e+02,1.031900e+04,12.000000
25%,64.250000,2242.500000,12.830000,12.770000,1.096135e+05,2.948452e+06,508.500000
50%,128.500000,4080.500000,25.925000,25.800000,2.456810e+05,9.637432e+06,1137.500000
75%,192.750000,4701.750000,57.150000,56.800000,7.564478e+05,2.609448e+07,2225.250000
max,257.000000,9411.000000,1081.600000,1089.000000,8.974221e+08,1.071074e+09,27539.000000


In [10]:
def extract_low_high(range_str):
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", range_str)
    if len(numbers) >= 2:
        values = [float(num) for num in numbers]
        return min(values), max(values)
    elif len(numbers) == 1:
        return float(numbers[0]), float(numbers[0])
    else:
        return None, None

# Apply the function to extract values and create new columns
data[['52_Week_Low', '52_Week_High']] = data['52 Weeks Range'].apply(lambda x: extract_low_high(x)).apply(pd.Series)
problematic_rows = data[data['52_Week_Low'] > data['52_Week_High']]
if not problematic_rows.empty:
    print("Problematic rows found:")
    print(problematic_rows[['Company Name', '52 Weeks Range', '52_Week_Low', '52_Week_High']])
else:
    print("All values are parsed correctly.")

All values are parsed correctly.


In [11]:
# Display column names
print("Available columns:")
print(data.columns.tolist())

Available columns:
['Unnamed: 0', 'Stock_Id', 'Company Name', '52 Weeks Range', 'Par Value', 'Price', 'Change', 'Change (%)', 'Prev. Close', 'Volume', 'Turnover', 'Transactions', '52_Week_Low', '52_Week_High']


In [12]:
# Example with possible alternative column names
try:
    # Try different possible column names
    if '52 Week High' in data.columns:
        data['Price_Range'] = data['52 Week High'] - data['52 Week Low']
    elif '52Week High' in data.columns:
        data['Price_Range'] = data['52Week High'] - data['52Week Low']
    elif '52_Week_High' in data.columns:
        data['Price_Range'] = data['52_Week_High'] - data['52_Week_Low']
    else:
        print("Please provide the correct column names from your data:")
        print(data.columns.tolist())
except Exception as e:
    print(f"Error: {e}")

In [13]:
data['Price_Range'] = data['52_Week_High'] - data['52_Week_Low']

In [14]:
def clean_company_name(name):
    clean_name = re.split(r'\sCompanies\b', name, maxsplit=1)[0]
    return clean_name.strip()   
data['Company Name'] = data['Company Name'].apply(clean_company_name)

In [15]:
data['Volume'] = pd.to_numeric(data['Volume'], errors='coerce')
data['Turnover'] = pd.to_numeric(data['Turnover'], errors='coerce')
data['Transactions'] = pd.to_numeric(data['Transactions'], errors='coerce')

In [16]:
def remove_parentheses(value):
    if isinstance(value, str):
        return float(value.strip('()'))
    return float(value)
data['Change'] = data['Change'].apply(remove_parentheses)

In [17]:
data = data.drop(columns=['Unnamed: 0'])

In [18]:
display(data.head(0))

,Stock_Id,Company Name,52 Weeks Range,Par Value,Price,Change,Change (%),Prev. Close,Volume,Turnover,Transactions,52_Week_Low,52_Week_High,Price_Range


# Exploratory Data Analysis (EDA)
 This section provides visual insights into the stock market data to better understand trends, distributions, and key metrics. 

In [19]:
top_10_data = data.sort_values(by='Price', ascending=False).head(10)
fig = px.bar(top_10_data, x='Company Name', y='Price', title='Top 10 Stock Prices by Company')
fig.show()

In [20]:
fig1 = px.scatter(data, x='Stock_Id', y='Price', color='Change', title='Stock Price and Change Over Time', labels={'Stock_Id': 'Stock ID', 'Price': 'Price'})
fig1.show()

In [21]:
largest_price_range = data.sort_values(by='Price_Range', ascending=False).head()
fig = px.bar(
    largest_price_range,
    x='Company Name',
    y='Price_Range',
    title='Companies with Largest Price Range',
    color='Price_Range',  
    color_continuous_scale='Plasma'  
)
fig.show()

In [22]:
top_volume_data = data.sort_values(by='Volume', ascending=False).head(10)
fig = px.bar(
    top_volume_data,
    x='Company Name',
    y='Volume',
    title='Top 10 Companies by Trading Volume',
    color='Volume',
    color_continuous_scale='Cividis'
)
fig.show()

In [23]:
top_10_data = data.sort_values(by='Price_Range', ascending=False).head(10)
fig = px.line(
    top_10_data,
    x='Company Name',
    y='Price_Range',
    title='Top 10 Companies by Price Range',
    markers=True
)
fig.update_layout(xaxis=dict(tickangle=45))
fig.show()

In [24]:
fig_low = px.histogram(
    data,
    x='52_Week_Low',
    nbins=30,
    title='Distribution of 52-Week Low Prices',
    color_discrete_sequence=['blue']
)
fig_low.show()
fig_high = px.histogram(
    data,
    x='52_Week_High',
    nbins=30,
    title='Distribution of 52-Week High Prices',
    color_discrete_sequence=['green']
)
fig_high.show()

In [25]:
fig = px.scatter(
    data,
    x='Price',
    y='Turnover',
    size='Volume',
    color='Company Name',
    title='Price vs. Turnover with Volume as Bubble Size',
    hover_data=['Price', 'Turnover', 'Volume'],
    size_max=40  
)
fig.update_xaxes(type='log')  
fig.update_yaxes(type='log')  
fig.show()

# Analytical Questions and Insights
This section of the notebook is dedicated to answering key questions about stock market data through a series of structured analyses.

# 1. Which company has the highest current stock price?

In [26]:
highest_price_company = data.loc[data['Price'].idxmax()]
print("Company with the highest current stock price:", highest_price_company['Company Name'])

Company with the highest current stock price: ELM


# 2. Which company has experienced the most significant change in stock price?

In [27]:
most_significant_change_company = data.loc[data['Change'].abs().idxmax()]
print("Company with the most significant change in stock price:", most_significant_change_company['Company Name'])

Company with the most significant change in stock price: ACWA POWER


# 3. How do companies with high transaction volumes correlate with price stability or volatility?

In [28]:
data['Volatility'] = data['Change (%)'].abs()
high_transaction_companies = data.sort_values(by='Transactions', ascending=False).head(5)
print("Top 5 companies with highest transactions and their volatility:\n", high_transaction_companies[['Company Name', 'Transactions', 'Volatility']])

TypeError: bad operand type for abs(): 'str'

# 4. Which company has the smallest price range over the 52 weeks?

In [29]:
data['Price_Range'] = data['52_Week_High'] - data['52_Week_Low']
smallest_range_company = data.loc[data['Price_Range'].idxmin()]
print("Company with the smallest 52-week price range:", smallest_range_company['Company Name'])

Company with the smallest 52-week price range: ALINMA SUKUK


# 5. Are companies with the highest turnovers also experiencing the most significant price changes?

In [30]:
high_turnover_companies = data.sort_values(by='Turnover', ascending=False).head(5)
print("Top 5 companies with highest turnovers and their price changes:\n", high_turnover_companies[['Company Name', 'Turnover', 'Change (%)']])

Top 5 companies with highest turnovers and their price changes:
      Company Name    Turnover Change (%)
0    SAUDI ARAMCO  1071073603     1.63 %
156       ALRAJHI   713078290     1.98 %
159           SNB   494103789     0.00 %
165        ALBAHA   301206844     0.00 %
158        ALINMA   292974889     0.35 %


# 5. Which company is closest to its 52-week high, indicating a potential bullish trend?

In [57]:
data['Distance_to_High'] = data['52_Week_High'] - data['Price']
closest_to_high_company = data.loc[data['Distance_to_High'].idxmin()]
print("Company closest to its 52-week high:", closest_to_high_company['Company Name'])

Company closest to its 52-week high: ELM


# 6.  Which company has the largest percentage difference between its current price and its 52-week high?

In [58]:
data['Percentage_Below_High'] = ((data['52_Week_High'] - data['Price']) / data['52_Week_High']) * 100
largest_percentage_difference = data.loc[data['Percentage_Below_High'].idxmax()]
print("Company with the largest percentage difference from its 52-week high:", largest_percentage_difference['Company Name'])

Company with the largest percentage difference from its 52-week high: ARABIAN DRILLING


# 7.  Which companies have positive price changes and also high turnover, indicating strong market activity?

In [59]:
positive_change_high_turnover = data[(data['Change'] > 0) & (data['Turnover'] > data['Turnover'].median())]
print("Companies with positive price changes and high turnover:\n", positive_change_high_turnover[['Company Name', 'Price', 'Change', 'Turnover']])

Companies with positive price changes and high turnover:
          Company Name    Price  Change    Turnover
0        SAUDI ARAMCO    28.00    0.45  1071073603
1    ARABIAN DRILLING   110.20    1.40    23366519
2                ADES    19.74    0.06    17988539
3        PETRO RABIGH     8.37    0.13    15386263
5             ALDREES   135.40    1.20    31570475
..                ...      ...     ...         ...
239         SOLUTIONS   262.60    2.20    28398448
240               ELM  1081.60    7.40    35554932
241                2P    14.64    0.22    10036058
242       ALMAJED OUD   152.60    1.40    34575085
257            SAB HK    10.62    1.18    35209761

[119 rows x 4 columns]


# 8. What is the average price of stocks for companies that have a par value of 10?

In [60]:
average_price_par_10 = data[data['Par Value'] == 10]['Price'].mean()
print("Average price of stocks with par value of 10:", average_price_par_10)

Average price of stocks with par value of 10: 66.008


# 9. Which company has the highest percentage gained , and does it have high or low transaction volume?

In [61]:
highest_percentage_gain = data.loc[data['Change (%)'].idxmax()]
print("Company with the highest percentage gained:", highest_percentage_gain['Company Name'])
print("Transaction volume:", highest_percentage_gain['Transactions'])

Company with the highest percentage gained: RASAN
Transaction volume: 6851


# 10. How does the change in stock price correlate with the turnover and transaction volumes?

In [62]:
correlation_change_turnover = data['Change'].corr(data['Turnover'])
correlation_change_transactions = data['Change'].corr(data['Transactions'])
print("Correlation between price change and turnover:", correlation_change_turnover)
print("Correlation between price change and transactions:", correlation_change_transactions)

Correlation between price change and turnover: 0.0832928468622529
Correlation between price change and transactions: 0.14248364353973622


# 11. Are there any companies with a high price change but low transaction volume, indicating potential market inefficiencies?

In [63]:
high_change_low_volume = data[(data['Change'].abs() > data['Change'].abs().mean()) & (data['Transactions'] < data['Transactions'].mean())]
print("Companies with high price change but low transaction volume:\n", high_change_low_volume[['Company Name', 'Change', 'Transactions']])

Companies with high price change but low transaction volume:
          Company Name  Change  Transactions
1    ARABIAN DRILLING    1.40          1700
5             ALDREES    1.20          1711
10               AMAK    1.60          1400
34             ZOUJAJ    1.00           879
44              QACCO    1.00          1349
45               SPCC    0.70          1368
47              EPCCO    0.70          1181
52      RIYADH CABLES    2.40          1859
55   ASTRA INDUSTRIAL    1.20          1816
60          ALBABTAIN    0.80          1564
61        SAUDI CABLE    2.40          1176
64            ALOMRAN    0.95           727
68            CATRION    2.20          1430
83             NASEEJ    3.30          1777
90                DWF    3.20          1068
93               NCLE    5.80           587
96             ALAMAR    2.20          1514
100          ALARABIA    2.40          1249
105             SASCO    1.10           762
113             NAHDI    1.40          1606
114           

# 12. Which companies have a low 52-week price range but significant daily price changes, indicating recent volatility?

In [64]:
low_range_high_volatility = data[(data['Price_Range'] < data['Price_Range'].mean()) & (data['Change (%)'].abs() > data['Change (%)'].mean())]
print("Companies with a low 52-week range but significant daily changes:\n", low_range_high_volatility[['Company Name', 'Price_Range', 'Change (%)']])

TypeError: bad operand type for abs(): 'str'

# 13. Which company has the highest volume-to-turnover ratio?

In [65]:
data['Volume_to_Turnover_Ratio'] = data['Volume'] / data['Turnover']
highest_ratio_company = data.loc[data['Volume_to_Turnover_Ratio'].idxmax()]
print("Company with the highest volume-to-turnover ratio:", highest_ratio_company['Company Name'])

Company with the highest volume-to-turnover ratio: ALBAHA


# 14. How many companies have experienced a price increase despite a relatively low transaction volume?

In [39]:
low_volume_price_increase = data[(data['Change'] > 0) & (data['Transactions'] < data['Transactions'].median())]
count_low_volume_increase = len(low_volume_price_increase)
print("Number of companies with a price increase despite low transaction volume:", count_low_volume_increase)

Number of companies with a price increase despite low transaction volume: 118


# 15. Which company has the most consistent price performance, showing the smallest average daily price change?

In [40]:
data['Daily_Change_Abs'] = data['Change'].abs()
most_consistent_company = data.loc[data['Daily_Change_Abs'].idxmin()]
print("Company with the most consistent price performance:", most_consistent_company['Company Name'])

Company with the most consistent price performance: NAMA CHEMICALS
